<a href="https://colab.research.google.com/github/CoolHero83/Prediccion-Transacciones-Fraudulentas/blob/main/ModeloRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# prompt: leer el archi csv


import pandas as pd
import pathlib

RUTA = pathlib.Path(__vsc_ipynb_file__).parent.parent # type: ignore

df = pd.read_csv(f"{RUTA}/Data/Data2.csv", encoding='latin1', sep=',', on_bad_lines='skip')
df.shape

(1999889, 26)

In [11]:
df.head(5)

,Amount,Is_Fraud,IngresoAnual,DebitoTotal,FICO_Score,Year,Month,Time,LimiteCredito,Errors_Insufficient Balance,...,Dia_Semana_MiÃ©rcoles,Dia_Semana_SÃ¡bado,Dia_Semana_Viernes,BancoCredito_Amex,BancoCredito_Discover,BancoCredito_Mastercard,BancoCredito_Visa,TipoTarjeta_Credit,TipoTarjeta_Debit,TipoTarjeta_Debit (Prepaid)
0,0.870166,0,1.058518,-0.269604,0.217595,-4.105101,-1.592169,0.905811,1.531309,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.744696,0,1.058518,-0.269604,0.217595,-4.105101,-1.592169,-1.230862,1.531309,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.060533,0,1.058518,-0.269604,0.217595,-4.105101,-1.592169,-0.445224,1.531309,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.139404,0,1.058518,-0.269604,0.217595,-4.105101,-1.304044,-1.142108,1.531309,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.390713,0,1.058518,-0.269604,0.217595,-4.105101,-1.304044,-0.326885,1.531309,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [13]:
# Cargar datos
X = df.drop('Is_Fraud', axis=1)
y = df['Is_Fraud']


In [14]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [15]:
# Aplicar SMOTE para sobremuestreo de la clase minoritaria
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

In [16]:
# Entrenar un modelo de Random Forest
RdForest = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
RdForest.fit(X_train_sm, y_train_sm)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [17]:
# Predecir en el conjunto de prueba
y_pred = RdForest.predict(X_test)

In [18]:
# Evaluar el modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[599162     55]
 [   734     16]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    599217
           1       0.23      0.02      0.04       750

    accuracy                           1.00    599967
   macro avg       0.61      0.51      0.52    599967
weighted avg       1.00      1.00      1.00    599967



In [19]:
# Identificando el procentual de error
from sklearn import metrics
from sklearn.metrics import mean_squared_error

error = mean_squared_error(y_test, y_pred)
print("Error porcentual : %f" % (error*100))
print("Exactitud:",metrics.accuracy_score(y_test, y_pred))

Error porcentual : 0.131507
Exactitud: 0.9986849276710219


In [26]:
#poner a prueba el modelo clf

RUTA = pathlib.Path(__vsc_ipynb_file__).parent.parent # type: ignore
DatosPrueba = pd.read_csv(f"{RUTA}/Data/DatosVerificacion.csv", encoding='latin1', sep=',', on_bad_lines='skip')
DatosPrueba.shape

(10, 26)

In [27]:
XPrueba = DatosPrueba.drop('Is_Fraud', axis=1)
yPrueba = DatosPrueba['Is_Fraud']


In [28]:
#El conjunto de datos para prueba tiene 10 transacciones, 5 fraudulentas y 5 que no lo son, DatosPrueba =[1,1,1,1,1,0,0,0,0,,0]

Prediccion = RdForest.predict(XPrueba)

In [29]:
Prediccion

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)